Dependencies and Libraries





In [1]:
import numpy
from PIL import Image

In [2]:
!wget https://d8it4huxumps7.cloudfront.net/files/62babe062492f_images.zip  ##Source for the image dataset(Here 15 High Quality images)

--2022-07-03 06:24:33--  https://d8it4huxumps7.cloudfront.net/files/62babe062492f_images.zip
Resolving d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)... 108.159.224.34, 108.159.224.182, 108.159.224.42, ...
Connecting to d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)|108.159.224.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72470683 (69M) [application/zip]
Saving to: ‘62babe062492f_images.zip’

62babe062492f_image 100%[===================>]  69.11M  89.2MB/s    in 0.8s    

2022-07-03 06:24:33 (89.2 MB/s) - ‘62babe062492f_images.zip’ saved [72470683/72470683]



In [3]:
!unzip 62babe062492f_images.zip -d data

Archive:  62babe062492f_images.zip
   creating: data/Images/
  inflating: data/__MACOSX/._Images  
  inflating: data/Images/.DS_Store   
  inflating: data/__MACOSX/Images/._.DS_Store  
  inflating: data/Images/sample13.jpg  
  inflating: data/__MACOSX/Images/._sample13.jpg  
  inflating: data/Images/sample12.jpg  
  inflating: data/__MACOSX/Images/._sample12.jpg  
  inflating: data/Images/sample10.jpg  
  inflating: data/__MACOSX/Images/._sample10.jpg  
  inflating: data/Images/sample11.jpg  
  inflating: data/__MACOSX/Images/._sample11.jpg  
  inflating: data/Images/sample9.jpg  
  inflating: data/__MACOSX/Images/._sample9.jpg  
  inflating: data/Images/sample15.jpg  
  inflating: data/__MACOSX/Images/._sample15.jpg  
  inflating: data/Images/sample14.jpg  
  inflating: data/__MACOSX/Images/._sample14.jpg  
  inflating: data/Images/sample8.jpg  
  inflating: data/__MACOSX/Images/._sample8.jpg  
  inflating: data/Images/sample6.jpg  
  inflating: data/__MACOSX/Images/._sample6.jpg  
  

In [51]:
!pip install image-quality

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 146 kB 5.5 MB/s 


In [53]:
import imquality.brisque as brisque
from skimage import io, img_as_float

**Image** **Quality** **Assessment** **Using** **Brisque**

In [54]:
img = img_as_float(io.imread('/content/data/Images/sample1.jpg', as_gray=True))
score=brisque.score(img)
print(score)

/usr/local/lib/python3.7/dist-packages/imquality/brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)
/usr/local/lib/python3.7/dist-packages/imquality/brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


19.83849943878849


**Image** **Compression**

Using SVD For image compression
If A is the matrix of the original image then using SVD we have
A=UEV
U= unitary matrix
E= Rectangular Diagonal Matrix
V= Unitary Matrix

In [18]:
def openImage(imagePath):
    imOrig = Image.open(imagePath)
    im = numpy.array(imOrig)

    aRed = im[:, :, 0]
    aGreen = im[:, :, 1]
    aBlue = im[:, :, 2]

    return [aRed, aGreen, aBlue, imOrig]

In [19]:
def compressSingleChannel(channelDataMatrix, singularValuesLimit):
    uChannel, sChannel, vhChannel = numpy.linalg.svd(channelDataMatrix)
    aChannelCompressed = numpy.zeros((channelDataMatrix.shape[0], channelDataMatrix.shape[1]))
    k = singularValuesLimit

    leftSide = numpy.matmul(uChannel[:, 0:k], numpy.diag(sChannel)[0:k, 0:k])
    aChannelCompressedInner = numpy.matmul(leftSide, vhChannel[0:k, :])
    aChannelCompressed = aChannelCompressedInner.astype('uint8')
    return aChannelCompressed

In [20]:
aRed, aGreen, aBlue, originalImage = openImage('/content/data/Images/sample10.jpg') ##Enter The Path of your Image(One At a time)

In [21]:
imageWidth = 512
imageHeight = 512

In [22]:
singularValuesLimit = 160

In [23]:
aRedCompressed = compressSingleChannel(aRed, singularValuesLimit)
aGreenCompressed = compressSingleChannel(aGreen, singularValuesLimit)
aBlueCompressed = compressSingleChannel(aBlue, singularValuesLimit)

In [24]:
imr = Image.fromarray(aRedCompressed, mode=None)
img = Image.fromarray(aGreenCompressed, mode=None)
imb = Image.fromarray(aBlueCompressed, mode=None)


In [25]:
newImage = Image.merge("RGB", (imr, img, imb))
newImage.save('/content/compressed/new10.jpg')

In [26]:
mr = imageHeight
mc = imageWidth

In [27]:
originalSize = mr * mc * 3
compressedSize = singularValuesLimit * (1 + mr + mc) * 3

**Image Size Comparrison of Both the images**

In [28]:
print('original size:')
print(originalSize)

print('compressed size:')
print(compressedSize)

print('Ratio compressed size / original size:')
ratio = compressedSize * 1.0 / originalSize
print(ratio)

print('Compressed image size is ' + str(round(ratio * 100, 2)) + '% of the original image ')

original size:
786432
compressed size:
492000
Ratio compressed size / original size:
0.6256103515625
Compressed image size is 62.56% of the original image 


**Brisque for the Compressed Image**

In [55]:
img = img_as_float(io.imread('/content/compressed/new.jpg', as_gray=True))
score=brisque.score(img)
print(score)

/usr/local/lib/python3.7/dist-packages/imquality/brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)
/usr/local/lib/python3.7/dist-packages/imquality/brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


36.05348712342786
